# Publication Visualizations: Ergodic Insurance Analysis

Generates all publication-ready figures for the research paper.
Each chart is 300 DPI, colorblind-accessible (Okabe-Ito palette),
and styled for LaTeX/Computer Modern compatibility.

**Prerequisites:** Run `1. process_vol_sim_results.ipynb` to build `cache/dashboard_cache.pkl`.

See `VISUAL_SPECIFICATION.md` for the full design system.

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import matplotlib.colors as mcolors
from matplotlib.gridspec import GridSpec
import numpy as np
import pickle
from pathlib import Path

# === User-configurable target ===
TARGET_CAP = 5_000_000
TARGET_ATR = 1.0

# Publication rcParams (see VISUAL_SPECIFICATION.md)
plt.rcParams.update({
    'font.family': 'serif',
    'font.serif': ['CMU Serif', 'Computer Modern', 'Times New Roman', 'DejaVu Serif'],
    'font.size': 10, 'axes.titlesize': 11, 'axes.labelsize': 10,
    'xtick.labelsize': 9, 'ytick.labelsize': 9, 'legend.fontsize': 9,
    'legend.framealpha': 0.9, 'legend.edgecolor': '#CCCCCC',
    'figure.dpi': 150, 'savefig.dpi': 300,
    'savefig.bbox': 'tight', 'savefig.pad_inches': 0.1,
    'axes.spines.top': False, 'axes.spines.right': False,
    'axes.grid': True, 'axes.axisbelow': True,
    'grid.alpha': 0.3, 'grid.linestyle': '--', 'grid.linewidth': 0.5,
    'text.usetex': False, 'mathtext.fontset': 'cm',
})

# Okabe-Ito colorblind-safe palette
PAL = {
    'blue': '#0072B2', 'orange': '#E69F00', 'green': '#009E73',
    'vermillion': '#D55E00', 'purple': '#CC79A7', 'sky': '#56B4E9',
    'yellow': '#F0E442', 'gray': '#999999', 'dark': '#333333',
}
DED_COLORS = {0: PAL['blue'], 100_000: PAL['sky'], 250_000: PAL['orange'], 500_000: PAL['vermillion']}
DED_LABELS = {0: '$0', 100_000: '$100K', 250_000: '$250K', 500_000: '$500K'}

# Extended maps including No Insurance
DED_COLORS_EXT = {**DED_COLORS, 'noins': PAL['gray']}
DED_LABELS_EXT = {**DED_LABELS, 'noins': 'No Insurance'}

# Line widths
LW_EM, LW_STD, LW_REF, LW_THIN = 2.5, 1.5, 1.0, 0.3

OUTPUT = Path('output/publication')
OUTPUT.mkdir(parents=True, exist_ok=True)

def fmt(n):
    if abs(n) >= 1e9: return f'${n/1e9:.0f}B'
    if abs(n) >= 1e6: return f'${n/1e6:.0f}M'
    if abs(n) >= 1e3: return f'${n/1e3:.0f}K'
    return f'${n:.0f}'

def save(fig, name):
    p = OUTPUT / f'{name}.png'
    fig.savefig(p, dpi=300, facecolor='white')
    print(f'Saved: {p}')

# DECOMP and PRICING are built dynamically in the cache-loading cell below

In [ ]:
import gc

# Load dashboard cache
with open('cache/dashboard_cache.pkl', 'rb') as f:
    cache = pickle.load(f)
configs = cache['configs']
crn_pairs = cache['crn_pairs']
param_values = cache['param_values']
RESULTS_DIR = Path(cache['results_dir'])

def find_key(cap, atr, ded=None, noins=False):
    for k, c in configs.items():
        if c['Cap'] == cap and c['ATR'] == atr:
            if noins and c.get('NOINS'): return k
            if not noins and not c.get('NOINS') and c.get('Ded') == ded: return k
    return None

def ruin_val(cfg, yr):
    rp = cfg['ruin_probability']
    for k, v in rp.items():
        if int(k) == yr: return v
    return None

def get_per_sim_ruin(key, min_trailing=2):
    """Identify ruined sims from trailing zeros in annual_losses.

    The simulation breaks out of the year loop on ruin, leaving trailing zeros
    in pre-allocated loss arrays. Sims with >= min_trailing consecutive trailing
    zero years are classified as ruined. This is needed because the simulation
    tracks ruin via equity <= 0 OR payment_burden > 80%, but per-sim ruin flags
    are not stored in SimulationResults — only the aggregate ruin_probability.
    Using min_trailing=2 matches the aggregate ruin_probability within ~0.4%.
    """
    path = RESULTS_DIR / f'{key}.pkl'
    with open(path, 'rb') as f:
        data = pickle.load(f)
    losses = data.annual_losses
    n_sims, n_years = losses.shape
    last_active = np.full(n_sims, -1, dtype=int)
    for yr in range(n_years - 1, -1, -1):
        active = (losses[:, yr] != 0) & (last_active == -1)
        last_active[active] = yr
    ruined = last_active < (n_years - min_trailing)
    del data
    gc.collect()
    return ruined

# Load growth_rates and final_assets from pickles
sim = {}  # key -> {'gr': array, 'fa': array}
def load_sim(key):
    if key in sim: return sim[key]
    path = RESULTS_DIR / f'{key}.pkl'
    with open(path, 'rb') as f:
        data = pickle.load(f)
    sim[key] = {'gr': data.growth_rates.copy(), 'fa': data.final_assets.copy()}
    del data
    return sim[key]

# Preload key configs
print('Loading simulation arrays...')
for cap in param_values['Cap']:
    for atr in param_values['ATR']:
        k = find_key(cap, atr, noins=True)
        if k:
            load_sim(k)
            print(f'  NOINS Cap={cap/1e6:.0f}M ATR={atr}')
        for ded in param_values['Ded']:
            k = find_key(cap, atr, ded=ded)
            if k:
                load_sim(k)
                print(f'  Ins   Cap={cap/1e6:.0f}M ATR={atr} Ded={ded/1e3:.0f}K')
print(f'Loaded {len(sim)} configs')

# ---------------------------------------------------------------------------
# Dynamic DECOMP / PRICING builder
# ---------------------------------------------------------------------------
def build_decomp(cap, atr, ebitabl=0.125, lr=0.70, tax_rate=0.25,
                 retention_ratio=0.70, pricing_iters=50_000, seed=42):
    """Build DECOMP and PRICING dicts for any (Cap, ATR) pair."""
    from ergodic_insurance.loss_distributions import ManufacturingLossGenerator

    revenue = cap * atr
    DET_RATIO = 0.556  # g_det/g_naive, empirically stable across all configs

    generator = ManufacturingLossGenerator(
        attritional_params={
            "base_frequency": 2.85 * revenue / 10_000_000,
            "severity_mean": 40_000,
            "severity_cv": 0.8,
            "revenue_scaling_exponent": 1.0,
            "reference_revenue": revenue,
        },
        large_params={
            "base_frequency": 0.20 * revenue / 10_000_000,
            "severity_mean": 500_000,
            "severity_cv": 1.5,
            "revenue_scaling_exponent": 1.0,
            "reference_revenue": revenue,
        },
        catastrophic_params={
            "base_frequency": 0.02 * revenue / 10_000_000,
            "severity_xm": 5_000_000,
            "severity_alpha": 2.5,
            "revenue_scaling_exponent": 1.0,
            "reference_revenue": revenue,
        },
        seed=seed,
    )

    # Run pricing sim for each deductible
    deductibles = {'$0K': 0, '$100K': 100_000, '$250K': 250_000, '$500K': 500_000}
    policy_limit = 100_000_000_000
    expected_insured, expected_retained = {}, {}
    for label, ded in deductibles.items():
        total_insured = 0.0
        total_retained = 0.0
        for _ in range(pricing_iters):
            loss_events, _ = generator.generate_losses(duration=1, revenue=revenue)
            for ev in loss_events:
                ins = max(min(ev.amount, policy_limit) - ded, 0)
                total_insured += ins
                total_retained += ev.amount - ins
        expected_insured[label] = total_insured / pricing_iters
        expected_retained[label] = total_retained / pricing_iters

    # Build PRICING dict
    pricing = {
        'EBITABL': ebitabl, 'tax_rate': tax_rate,
        'retention_ratio': retention_ratio, 'base_LR': lr,
        'expected_losses': expected_insured,
        'expected_retained': expected_retained,
        'revenue': revenue,
    }

    # Build DECOMP for each deductible + No Insurance
    decomp = {}
    for label in list(deductibles) + ['No Ins']:
        # g_actual from dashboard cache
        if label == 'No Ins':
            k = find_key(cap, atr, noins=True)
        else:
            k = find_key(cap, atr, ded=deductibles[label])
        g_actual = configs[k]['growth_rate_mean'] * 10000

        # g_naive from analytical formula
        if label == 'No Ins':
            cost_rate = expected_insured['$0K'] / revenue  # full ground-up, no premium
        else:
            premium = expected_insured[label] / lr
            cost_rate = (premium + expected_retained[label]) / revenue
        g_naive = (ebitabl - cost_rate) * (1 - tax_rate) * retention_ratio * 10000

        g_det = g_naive * DET_RATIO
        decomp[label] = {
            'g_naive': round(g_naive, 1), 'g_det': round(g_det, 1),
            'g_actual': round(g_actual, 1),
            'tax_drag': round(g_naive - g_det, 1),
            'stoch_pen': round(g_det - g_actual, 1),
        }

    return decomp, pricing

# Build DECOMP for selected configuration
DECOMP, PRICING = build_decomp(TARGET_CAP, TARGET_ATR)
print(f'DECOMP for Cap=${TARGET_CAP/1e6:.0f}M, ATR={TARGET_ATR}:')
for label, d in DECOMP.items():
    print(f'  {label:>8}: g_naive={d["g_naive"]:6.1f}  g_det={d["g_det"]:6.1f}  '
          f'g_actual={d["g_actual"]:6.1f}  stoch_pen={d["stoch_pen"]:+6.1f}')

## 2. Optimal Deductible Heatmap
Core actionable result: which deductible maximizes time-average growth rate for each (Cap, ATR) combination.

In [ ]:
caps = param_values['Cap']
atrs = param_values['ATR']
deds = param_values['Ded']

# Build growth rate matrix and find optimal
opt_ded = np.full((len(caps), len(atrs)), np.nan)
opt_gr = np.full((len(caps), len(atrs)), np.nan)
worst_gr = np.full((len(caps), len(atrs)), np.nan)
penalty = np.full((len(caps), len(atrs)), np.nan)

for i, cap in enumerate(caps):
    for j, atr in enumerate(atrs):
        best_g, best_d, worst_g = -np.inf, None, np.inf
        for ded in deds:
            k = find_key(cap, atr, ded=ded)
            if k is None: continue
            g = configs[k]['growth_rate_mean']
            if g > best_g: best_g, best_d = g, ded
            if g < worst_g: worst_g = g
        if best_d is not None:
            opt_ded[i, j] = best_d
            opt_gr[i, j] = best_g
            worst_gr[i, j] = worst_g
            penalty[i, j] = (best_g - worst_g) * 10_000  # basis points

# Plot heatmap
fig, ax = plt.subplots(figsize=(5.5, 4.0))

# Map deductible values to indices for coloring
ded_to_idx = {d: i for i, d in enumerate(deds)}
color_matrix = np.array([[ded_to_idx.get(opt_ded[i, j], -1) for j in range(len(atrs))] for i in range(len(caps))])
cmap = mcolors.ListedColormap([DED_COLORS[d] for d in deds])
bounds = np.arange(-0.5, len(deds) + 0.5, 1)
norm = mcolors.BoundaryNorm(bounds, cmap.N)

im = ax.imshow(color_matrix, cmap=cmap, norm=norm, aspect='auto')

# Annotate cells
for i in range(len(caps)):
    for j in range(len(atrs)):
        if np.isnan(opt_ded[i, j]):
            ax.text(j, i, 'N/A', ha='center', va='center', fontsize=9, color=PAL['gray'])
        else:
            txt_color = 'white' if color_matrix[i, j] in [0, 3] else PAL['dark']
            ax.text(j, i - 0.15, DED_LABELS[int(opt_ded[i, j])],
                    ha='center', va='center', fontsize=10, fontweight='bold', color=txt_color)
            ax.text(j, i + 0.2,
                    f'g*={opt_gr[i,j]:.4f}\n\u0394={penalty[i,j]:.0f} bps',
                    ha='center', va='center', fontsize=7.5, color=txt_color)

ax.set_xticks(range(len(atrs)))
ax.set_xticklabels([str(a) for a in atrs])
ax.set_yticks(range(len(caps)))
ax.set_yticklabels([fmt(c) for c in caps])
ax.set_xlabel('Asset Turnover Ratio')
ax.set_ylabel('Initial Capitalization')
ax.set_title('Optimal Deductible by Company Profile', fontsize=13, fontweight='bold', pad=12)

# Grid between cells
ax.set_xticks(np.arange(-0.5, len(atrs), 1), minor=True)
ax.set_yticks(np.arange(-0.5, len(caps), 1), minor=True)
ax.grid(which='minor', color='white', linewidth=2)
ax.grid(which='major', visible=False)
ax.tick_params(which='minor', size=0)

# Legend
from matplotlib.patches import Patch
legend_patches = [Patch(facecolor=DED_COLORS[d], label=DED_LABELS[d]) for d in deds]
ax.legend(handles=legend_patches, loc='upper left', bbox_to_anchor=(1.02, 1), title='Optimal Ded')

fig.tight_layout()
save(fig, 'optimal_deductible_heatmap')
plt.show()

## 3. Wealth Trajectory Fan Chart
Insured vs. uninsured wealth evolution over 50 years. Percentile envelopes
from surviving paths only; vermillion dots mark insolvency events where
liabilities exceeded assets (equity ≤ 0). Note: ruin means balance-sheet
insolvency, not total assets reaching zero.

In [ ]:
CAP, ATR, DED = 5_000_000, 1.0, 0
A0 = CAP
years = np.arange(51)
bands = [(5, 95, 0.10), (25, 75, 0.20), (40, 60, 0.35)]

def load_ruin_years(key):
    """Detect ruin year per sim from trailing zeros in annual_losses.
    Returns -1 for survivors, ruin year (0-indexed) for ruined paths."""
    path = RESULTS_DIR / f'{key}.pkl'
    with open(path, 'rb') as f:
        data = pickle.load(f)
    losses = data.annual_losses
    n_sims, n_years = losses.shape
    last_active = np.full(n_sims, -1, dtype=int)
    for yr in range(n_years - 1, -1, -1):
        mask = (losses[:, yr] != 0) & (last_active == -1)
        last_active[mask] = yr
    ruined = last_active < (n_years - 2)
    ruin_year = np.where(ruined, last_active + 1, -1)
    del data
    gc.collect()
    return ruin_year

fig, axes = plt.subplots(1, 2, figsize=(7.0, 4.0), sharey=True)

for ax_idx, (label, noins_flag, ded_val, base_color) in enumerate([
    (f'Insured (Ded {DED_LABELS[DED]})', False, DED, PAL['blue']),
    ('No Insurance', True, None, PAL['orange']),
]):
    ax = axes[ax_idx]
    k = find_key(CAP, ATR, ded=ded_val, noins=noins_flag)
    if k is None:
        ax.set_title(f'{label} (no data)'); continue

    g = sim[k]['gr']
    fa = sim[k]['fa']
    ry = load_ruin_years(k)

    survived = ry == -1
    ruined = ry >= 0
    n_ruined = ruined.sum()
    ruin_pct = n_ruined / len(g)
    g_surv = g[survived]

    rng = np.random.default_rng(42)

    # --- Surviving paths: spaghetti + fan ---
    n_surv_sample = min(300, survived.sum())
    surv_idx = rng.choice(np.where(survived)[0], size=n_surv_sample, replace=False)
    for si in surv_idx:
        y = A0 * np.exp(g[si] * years)
        ax.plot(years, y, color=base_color, alpha=0.04, lw=LW_THIN)

    for lo, hi, alpha in bands:
        g_lo = np.percentile(g_surv, lo)
        g_hi = np.percentile(g_surv, hi)
        ax.fill_between(years, A0 * np.exp(g_lo * years),
                        A0 * np.exp(g_hi * years), color=base_color, alpha=alpha)

    ax.plot(years, A0 * np.exp(np.median(g_surv) * years),
            color=base_color, lw=LW_EM, label='Median (survivors)')

    # --- Ruined paths: terminated trajectories + insolvency dots ---
    if n_ruined > 50:
        n_ruin_sample = min(150, n_ruined)
        ruin_idx = rng.choice(np.where(ruined)[0], size=n_ruin_sample, replace=False)
        ruin_x, ruin_y = [], []
        for si in ruin_idx:
            ruin_yr = ry[si]
            if ruin_yr <= 0: continue
            g_to_ruin = np.log(float(fa[si]) / A0) / ruin_yr
            t_path = np.arange(ruin_yr + 1)
            y_path = A0 * np.exp(g_to_ruin * t_path)
            ax.plot(t_path, y_path, color=PAL['vermillion'], alpha=0.12, lw=0.5)
            ruin_x.append(ruin_yr)
            ruin_y.append(y_path[-1])

        ax.scatter(ruin_x, ruin_y, color=PAL['vermillion'], s=4, alpha=0.3,
                   zorder=4, label=f'Insolvency event ({ruin_pct:.0%})')

        median_ry = np.median(ry[ruined])
        ax.text(0.97, 0.05,
                f'{ruin_pct:.1%} insolvent\nmedian ruin year: {median_ry:.0f}',
                transform=ax.transAxes, fontsize=8, ha='right', va='bottom',
                color=PAL['vermillion'], fontweight='bold',
                bbox=dict(boxstyle='round,pad=0.3', fc='white', alpha=0.9,
                          ec=PAL['vermillion'], lw=0.8))
    elif ruin_pct > 0:
        ax.text(0.97, 0.05, f'{ruin_pct:.2%} insolvent',
                transform=ax.transAxes, fontsize=8, ha='right', va='bottom',
                color=PAL['gray'])

    ax.set_yscale('log')
    ax.set_ylim(1e6, None)
    ax.set_title(label, fontsize=11)
    ax.set_xlabel('Year')
    ax.legend(loc='upper left', fontsize=7.5)

axes[0].set_ylabel('Total Assets ($)')
fig.suptitle(
    f'Wealth Trajectories: Insured vs Uninsured (Cap={fmt(CAP)}, ATR={ATR})',
    fontsize=13, fontweight='bold', y=1.02,
)
fig.tight_layout()
fig.savefig(fr'{OUTPUT}\wealth_fan_chart', dpi=300, facecolor='white')
plt.show()

## 4. Survival Curves
Fraction of companies still solvent at each evaluation year,
across deductible levels. Shows how ruin accumulates over time.

In [ ]:
CAP, ATR = 5_000_000, 1.0

fig, ax = plt.subplots(figsize=(5.5, 3.5))

# Plot survival for each deductible + NOINS
for ded in deds:
    k = find_key(CAP, ATR, ded=ded)
    if k is None: continue
    rp = configs[k]['ruin_probability']
    eval_years = sorted(rp.keys(), key=int)
    xs = [0] + [int(y) for y in eval_years]
    ys = [1.0] + [1.0 - rp[y] for y in eval_years]
    ax.plot(xs, ys, color=DED_COLORS[ded], lw=LW_EM, marker='o', ms=3,
            label=f'Ded {DED_LABELS[ded]}')

# NOINS baseline
k_ni = find_key(CAP, ATR, noins=True)
if k_ni:
    rp_ni = configs[k_ni]['ruin_probability']
    eval_years_ni = sorted(rp_ni.keys(), key=int)
    xs_ni = [0] + [int(y) for y in eval_years_ni]
    ys_ni = [1.0] + [1.0 - rp_ni[y] for y in eval_years_ni]
    ax.plot(xs_ni, ys_ni, color=PAL['gray'], lw=LW_STD, ls='--', marker='s', ms=3,
            label='No Insurance')

ax.set_xlabel('Year')
ax.set_ylabel('Survival Probability')
ax.set_title(
    f'Survival Curves by Deductible (Cap={fmt(CAP)}, ATR={ATR})',
    fontsize=13, fontweight='bold',
)
ax.set_ylim(None, 1.005)
ax.yaxis.set_major_formatter(mticker.PercentFormatter(1.0, decimals=1))
ax.legend(loc='lower left', fontsize=8)

fig.tight_layout()
save(fig, 'survival_curves')
plt.show()

## 6. Year-by-Year Growth Lift
Insurance value compounds over time. Shows cumulative log-wealth
advantage of insured vs. uninsured.

In [ ]:
CAP, ATR = 5_000_000, 1.0
years = np.arange(51)

k_ni = find_key(CAP, ATR, noins=True)
g_ni = sim[k_ni]['gr'] if k_ni else None

fig, ax = plt.subplots(figsize=(5.5, 3.5))

for ded in deds:
    k_ins = find_key(CAP, ATR, ded=ded)
    if k_ins is None or g_ni is None: continue
    g_ins = sim[k_ins]['gr']

    # Paired growth lift at each year
    # lift(t) = mean(g_ins_i * t - g_ni_i * t) = mean(delta_i) * t
    delta = g_ins - g_ni  # CRN-paired differences
    mean_delta = delta.mean()
    se_delta = delta.std() / np.sqrt(len(delta))

    lift = mean_delta * years
    lift_upper = (mean_delta + 1.96 * se_delta) * years
    lift_lower = (mean_delta - 1.96 * se_delta) * years

    ax.plot(years, lift, color=DED_COLORS[ded], lw=LW_EM,
            label=f'Ded {DED_LABELS[ded]} ({mean_delta*1e4:.1f} bps/yr)')
    ax.fill_between(years, lift_lower, lift_upper,
                    color=DED_COLORS[ded], alpha=0.10)

ax.axhline(0, color=PAL['gray'], ls=':', lw=LW_REF)
ax.set_xlabel('Year')
ax.set_ylabel('Cumulative Log-Wealth Advantage')
ax.set_title(
    f'Insurance Value Compounds Over Time (Cap={fmt(CAP)}, ATR={ATR})',
    fontsize=12, fontweight='bold',
)
ax.legend(loc='upper left', fontsize=8)

fig.tight_layout()
fig.savefig(fr'{OUTPUT}\year_by_year_growth_lift', dpi=300, facecolor='white')
plt.show()

In [ ]:
CAP_5, ATR_5 = 5_000_000, 1.0
CAP_10, ATR_10 = 10_000_000, 1.0
CAP_25, ATR_25 = 25_000_000, 1.0
years = np.arange(51)

k_ni_5 = find_key(CAP_5, ATR_5, noins=True)
g_ni_5 = sim[k_ni_5]['gr'] if k_ni_5 else None

k_ni_10 = find_key(CAP_10, ATR_10, noins=True)
g_ni_10 = sim[k_ni_10]['gr'] if k_ni_10 else None

k_ni_25 = find_key(CAP_25, ATR_25, noins=True)
g_ni_25 = sim[k_ni_25]['gr'] if k_ni_25 else None

fig, (ax_5, ax_10, ax_25) = plt.subplots(3, 1, figsize=(7.0, 10.0), sharex=True, sharey=True)

for ded in deds:
    k_ins_5 = find_key(CAP_5, ATR_5, ded=ded)
    k_ins_10 = find_key(CAP_10, ATR_10, ded=ded)
    k_ins_25 = find_key(CAP_25, ATR_25, ded=ded)
    if k_ins_5 is None or g_ni_5 is None: continue
    if k_ins_10 is None or g_ni_10 is None: continue
    if k_ins_25 is None or g_ni_25 is None: continue
    g_ins_5 = sim[k_ins_5]['gr']
    g_ins_10 = sim[k_ins_10]['gr']
    g_ins_25 = sim[k_ins_25]['gr']

    # Paired growth lift at each year
    # lift(t) = mean(g_ins_i * t - g_ni_i * t) = mean(delta_i) * t
    delta_5 = g_ins_5 - g_ni_5  # CRN-paired differences
    delta_10 = g_ins_10 - g_ni_10  # CRN-paired differences
    delta_25 = g_ins_25 - g_ni_25  # CRN-paired differences

    mean_delta_5 = delta_5.mean()
    mean_delta_10 = delta_10.mean()
    mean_delta_25 = delta_25.mean()

    se_delta_5 = delta_5.std() / np.sqrt(len(delta_5))
    se_delta_10 = delta_10.std() / np.sqrt(len(delta_10))
    se_delta_25 = delta_25.std() / np.sqrt(len(delta_25))

    lift_5 = mean_delta_5 * years
    lift_upper_5 = (mean_delta_5 + 1.96 * se_delta_5) * years
    lift_lower_5 = (mean_delta_5 - 1.96 * se_delta_5) * years

    lift_10 = mean_delta_10 * years
    lift_upper_10 = (mean_delta_10 + 1.96 * se_delta_10) * years
    lift_lower_10 = (mean_delta_10 - 1.96 * se_delta_10) * years

    lift_25 = mean_delta_25 * years
    lift_upper_25 = (mean_delta_25 + 1.96 * se_delta_25) * years
    lift_lower_25 = (mean_delta_25 - 1.96 * se_delta_25) * years

    ax_5.plot(years, lift_5, color=DED_COLORS[ded], lw=LW_EM,
            label=f'Ded {DED_LABELS[ded]} ({mean_delta_5*1e4:.1f} bps/yr)')
    ax_5.fill_between(years, lift_lower_5, lift_upper_5,
                    color=DED_COLORS[ded], alpha=0.10)
    
    ax_10.plot(years, lift_10, color=DED_COLORS[ded], lw=LW_EM,
            label=f'Ded {DED_LABELS[ded]} ({mean_delta_10*1e4:.1f} bps/yr)')
    ax_10.fill_between(years, lift_lower_10, lift_upper_10,
                    color=DED_COLORS[ded], alpha=0.10)
    
    ax_25.plot(years, lift_25, color=DED_COLORS[ded], lw=LW_EM,
            label=f'Ded {DED_LABELS[ded]} ({mean_delta_25*1e4:.1f} bps/yr)')
    ax_25.fill_between(years, lift_lower_25, lift_upper_25,
                    color=DED_COLORS[ded], alpha=0.10)

ax_5.axhline(0, color=PAL['gray'], ls=':', lw=LW_REF)
ax_5.set_xlabel('Year')
ax_5.set_ylabel('Cumulative Log-Wealth Advantage')
ax_5.set_title(
    f'{fmt(CAP_5)} Capitalization',
    fontsize=12, fontweight='bold',
)
ax_5.legend(loc='upper left', fontsize=8)

ax_10.axhline(0, color=PAL['gray'], ls=':', lw=LW_REF)
ax_10.set_xlabel('Year')
ax_10.set_ylabel('Cumulative Log-Wealth Advantage')
ax_10.set_title(
    f'{fmt(CAP_10)} Capitalization',
    fontsize=12, fontweight='bold',
)
ax_10.legend(loc='upper left', fontsize=8)

ax_25.axhline(0, color=PAL['gray'], ls=':', lw=LW_REF)
ax_25.set_xlabel('Year')
ax_25.set_ylabel('Cumulative Log-Wealth Advantage')
ax_25.set_title(
    f'{fmt(CAP_25)} Capitalization',
    fontsize=12, fontweight='bold',
)
ax_25.legend(loc='upper left', fontsize=8)

fig.suptitle(
    'Insurance Value Compounds Over Time',
    fontsize=13, fontweight='bold',
)

fig.tight_layout()
fig.savefig(fr'{OUTPUT}\year_by_year_growth_lift_facets', dpi=300, facecolor='white')
plt.show()

## 7. Peer Benchmark Is Wrong
Two companies with identical revenue but different balance sheets
should make fundamentally different insurance decisions.

In [ ]:
ATR_COMMON = 1.0
# Only $5M and $10M have full data (all deductibles + NOINS); $25M lacks NOINS baseline
caps_compare = [5_000_000, 10_000_000]
cap_colors = [PAL['blue'], PAL['sky']]

fig, axes = plt.subplots(1, 3, figsize=(7.0, 3.0))

# Panel 1: Growth rate by deductible for each Cap
ax = axes[0]
bar_w = 0.35
for ci, cap in enumerate(caps_compare):
    grs = []
    for ded in deds:
        k = find_key(cap, ATR_COMMON, ded=ded)
        grs.append(configs[k]['growth_rate_mean'] if k else np.nan)
    x = np.arange(len(deds)) + ci * bar_w
    ax.bar(x, grs, bar_w, color=cap_colors[ci],
           label=fmt(cap), edgecolor='white', lw=0.5)
ax.set_xticks(np.arange(len(deds)) + bar_w / 2)
ax.set_xticklabels([DED_LABELS[d] for d in deds], fontsize=8)
ax.set_ylabel('Growth Rate')
ax.set_title('Growth Rate', fontsize=10)
ax.legend(title='Cap', fontsize=8, title_fontsize=8)

# Panel 2: Ruin probability (year 50) - include NOINS baseline
ax = axes[1]
ded_labels_ext = [DED_LABELS[d] for d in deds] + ['No Ins.']
for ci, cap in enumerate(caps_compare):
    ruin_50s = []
    for ded in deds:
        k = find_key(cap, ATR_COMMON, ded=ded)
        ruin_50s.append(ruin_val(configs[k], 50) if k else np.nan)
    k_ni = find_key(cap, ATR_COMMON, noins=True)
    ruin_50s.append(ruin_val(configs[k_ni], 50) if k_ni else np.nan)
    x = np.arange(len(ded_labels_ext)) + ci * bar_w
    ax.bar(x, ruin_50s, bar_w, color=cap_colors[ci], edgecolor='white', lw=0.5)
ax.set_xticks(np.arange(len(ded_labels_ext)) + bar_w / 2)
ax.set_xticklabels(ded_labels_ext, fontsize=7)
ax.yaxis.set_major_formatter(mticker.PercentFormatter(1.0, decimals=0))
ax.set_title('Ruin Prob (50yr)', fontsize=10)

# Panel 3: Growth lift over no-insurance
ax = axes[2]
for ci, cap in enumerate(caps_compare):
    k_ni = find_key(cap, ATR_COMMON, noins=True)
    if k_ni is None: continue
    g_ni = configs[k_ni]['growth_rate_mean']
    lifts = []
    for ded in deds:
        k = find_key(cap, ATR_COMMON, ded=ded)
        lifts.append((configs[k]['growth_rate_mean'] - g_ni) * 1e4 if k else np.nan)
    x = np.arange(len(deds)) + ci * bar_w
    ax.bar(x, lifts, bar_w, color=cap_colors[ci], edgecolor='white', lw=0.5)
ax.set_xticks(np.arange(len(deds)) + bar_w / 2)
ax.set_xticklabels([DED_LABELS[d] for d in deds], fontsize=8)
ax.axhline(0, color=PAL['gray'], ls=':', lw=LW_REF)
ax.set_ylabel('Basis Points')
ax.set_title('Growth Lift vs No Ins.', fontsize=10)

fig.suptitle(
    f'Same Industry (ATR={ATR_COMMON}), Different Balance Sheets',
    fontsize=13, fontweight='bold', y=1.04,
)
fig.tight_layout()
save(fig, 'peer_benchmark_is_wrong')
plt.show()

## 8. Insurance Outcome Distribution ($0 Ded vs No Insurance)
Combined view: the distribution of per-path insurance value (left) and
key outcome metrics (right). Only the two extreme strategies are compared
since $0 deductible dominates all configurations.

In [ ]:
# === Insurance Outcome Distribution: $0 Ded vs No Insurance ===
CAP, ATR, DED = 5_000_000, 1.0, 0
k_ni = find_key(CAP, ATR, noins=True)
k_ins = find_key(CAP, ATR, ded=DED)
fa_ni = sim[k_ni]['fa']
fa_ins = sim[k_ins]['fa']

# Ruin detection
ruined_ni = get_per_sim_ruin(k_ni)
ruined_ins = get_per_sim_ruin(k_ins)

n = len(fa_ins)
both_ok = (~ruined_ins) & (~ruined_ni)
ins_saved = (~ruined_ins) & ruined_ni

# Delta for both-survived paths
delta_survived = np.log(fa_ins[both_ok].astype(float)) - np.log(fa_ni[both_ok].astype(float))

# Outperformance across ALL paths
outperform_all = fa_ins > fa_ni
delta_all = fa_ins.astype(float) - fa_ni.astype(float)
helps = delta_all > 0
avg_gain = delta_all[helps].mean()
avg_loss = abs(delta_all[~helps].mean())
ratio = avg_gain / avg_loss

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(7.0, 4.2),
                                gridspec_kw={'width_ratios': [1.2, 1]})

# --- Panel A: Distribution of log-wealth advantage ---
bins = np.linspace(-0.8, 3.0, 120)
bin_centers = (bins[:-1] + bins[1:]) / 2
counts, _ = np.histogram(delta_survived, bins=bins)
density = counts / (counts.sum() * (bins[1] - bins[0]))
colors_hist = [PAL['green'] if bc > 0 else PAL['vermillion'] for bc in bin_centers]

ax1.bar(bin_centers, density, width=bins[1]-bins[0], color=colors_hist,
        edgecolor='none', alpha=0.75)
ax1.axvline(delta_survived.mean(), color=PAL['dark'], lw=LW_STD, ls='-',
            label=f'Mean = {delta_survived.mean():.2f} (1.5\u00d7 wealth)')
ax1.axvline(np.median(delta_survived), color=PAL['dark'], lw=LW_STD, ls='--',
            label=f'Median = {np.median(delta_survived):.2f} (1.4\u00d7 wealth)')
ax1.axvline(0, color=PAL['gray'], ls=':', lw=LW_REF)

# I think this really measures the average survival wealth with positive delta, not % survival
frac_pos = (delta_survived > 0).mean()
# ax1.text(0.97, 0.95,
#          f'{frac_pos:.0%} of surviving paths\nend up wealthier with insurance',
#          transform=ax1.transAxes, fontsize=8, ha='right', va='top',
#          bbox=dict(boxstyle='round,pad=0.3', fc='#D4EDDA', alpha=0.8,
#                    ec=PAL['green'], lw=0.5))

ax1.set_xlabel('Log-Wealth Advantage (insured \u2212 uninsured)')
ax1.set_ylabel('Density')
ax1.set_title('A. Distribution of Insurance Value\n(paths where both survived)',
              fontsize=10, fontweight='bold')
ax1.legend(fontsize=7.5, loc='center right')

# --- Panel B: Key metrics scorecard ---
ax2.set_xlim(0, 1)
ax2.set_ylim(-1, 5.5)
ax2.axis('off')

metrics = [
    ('Saved from ruin', f'{ins_saved.mean():.1%}', PAL['green'],
     f'{ins_saved.sum():,} of {n:,} paths'),
    ('Wealthier at year 50', f'{outperform_all.mean():.1%}', PAL['blue'],
     f'across all 250K paired simulations'),
    ('Avg gain when helps', f'+{fmt(avg_gain)}', PAL['green'],
     f'{outperform_all.sum():,} paths ({outperform_all.mean():.0%})'),
    ('Avg loss when hurts', f'\u2212{fmt(avg_loss)}', PAL['vermillion'],
     f'{(~outperform_all).sum():,} paths ({(~outperform_all).mean():.0%})'),
]

for i, (label, value, color, detail) in enumerate(metrics):
    y = 4.5 - i * 1.25
    ax2.text(0.0, y, value, fontsize=16, fontweight='bold', color=color,
             va='center', ha='left')
    ax2.text(0.0, y - 0.35, label, fontsize=9, color=PAL['dark'],
             va='center', ha='left')
    ax2.text(0.0, y - 0.58, detail, fontsize=7, color=PAL['gray'],
             va='center', ha='left')

ax2.text(0.5, -0.5,
         f'Gain/Loss Asymmetry: {ratio:.1f}\u00d7',
         fontsize=12, fontweight='bold', color=PAL['blue'],
         ha='center', va='center',
         bbox=dict(boxstyle='round,pad=0.5', fc='#E8F0FE', alpha=0.9,
                   ec=PAL['blue'], lw=1.0))

ax2.set_title('B. The Asymmetric Bet\n(\\$0 Ded vs No Insurance)',
              fontsize=10, fontweight='bold')

fig.suptitle(
    'Guaranteed Cost Insurance: Heads You Win Big, Tails You Lose Small',
    fontsize=13, fontweight='bold', y=1.02,
)
fig.tight_layout()
fig.savefig(fr'{OUTPUT}\insurance_outcome_distribution', dpi=300, facecolor='white')
plt.show()

In [ ]:
# === Insurance Outcome Distribution: $0 Ded vs No Insurance ===
CAP, ATR, DED = 5_000_000, 1.0, 0
k_ni = find_key(CAP, ATR, noins=True)
k_ins = find_key(CAP, ATR, ded=DED)
fa_ni = sim[k_ni]['fa']
fa_ins = sim[k_ins]['fa']

# Ruin detection
ruined_ni = get_per_sim_ruin(k_ni)
ruined_ins = get_per_sim_ruin(k_ins)

n = len(fa_ins)
both_ok = (~ruined_ins) & (~ruined_ni)
ins_saved = (~ruined_ins) & ruined_ni

# Delta for both-survived paths
delta_survived = np.log(fa_ins[both_ok].astype(float)) - np.log(fa_ni[both_ok].astype(float))

# Outperformance across ALL paths
outperform_all = fa_ins > fa_ni
delta_all = fa_ins.astype(float) - fa_ni.astype(float)
helps = delta_all > 0
avg_gain = delta_all[helps].mean()
avg_loss = abs(delta_all[~helps].mean())
ratio = avg_gain / avg_loss

fig, ax1 = plt.subplots(figsize=(5.5, 4.0))

# --- Panel A: Distribution of log-wealth advantage ---
bins = np.linspace(-0.8, 3.0, 120)
bin_centers = (bins[:-1] + bins[1:]) / 2
counts, _ = np.histogram(delta_survived, bins=bins)
density = counts / (counts.sum() * (bins[1] - bins[0]))
colors_hist = [PAL['green'] if bc > 0 else PAL['vermillion'] for bc in bin_centers]

ax1.bar(bin_centers, density, width=bins[1]-bins[0], color=colors_hist,
        edgecolor='none', alpha=0.75)
ax1.axvline(delta_survived.mean(), color=PAL['dark'], lw=LW_STD, ls='-',
            label=f'Mean = {delta_survived.mean():.2f} ({np.exp(delta_survived.mean()):.1f} \u00d7 wealth)')
ax1.axvline(np.median(delta_survived), color=PAL['dark'], lw=LW_STD, ls='--',
            label=f'Median = {np.median(delta_survived):.2f} ({np.exp(np.median(delta_survived)):.1f} \u00d7 wealth)')
ax1.axvline(0, color=PAL['gray'], ls=':', lw=LW_REF)

# frac_pos = (delta_survived > 0).mean()
# ax1.text(0.97, 0.95,
#          f'{frac_pos:.0%} of surviving paths\nend up wealthier with insurance',
#          transform=ax1.transAxes, fontsize=8, ha='right', va='top',
#          bbox=dict(boxstyle='round,pad=0.3', fc='#D4EDDA', alpha=0.8,
#                    ec=PAL['green'], lw=0.5))

ax1.set_xlabel('Log-Wealth Advantage (insured \u2212 uninsured)')
ax1.set_ylabel('Density')
ax1.set_title('Distribution of Guaranteed Cost Insurance Value\n(paths where both survived)',
              fontsize=10, fontweight='bold')
ax1.legend(fontsize=7.5, loc='best')

fig.tight_layout()
fig.savefig(fr'{OUTPUT}\insurance_outcome_distribution_basic', dpi=300, facecolor='white')
plt.show()

## 9. Cost of Getting It Wrong
Growth penalty (in basis points) for choosing a suboptimal deductible.

In [ ]:
fig, ax = plt.subplots(figsize=(7.0, 3.5))
ax.axis('off')

# Build table data
row_labels = []
table_data = []
cell_colors = []

for cap in caps:
    for atr in atrs:
        row_labels.append(f'{fmt(cap)}, ATR={atr}')
        row_grs = []
        for ded in deds:
            k = find_key(cap, atr, ded=ded)
            row_grs.append(configs[k]['growth_rate_mean'] if k else np.nan)

        best_g = np.nanmax(row_grs)
        worst_g = np.nanmin(row_grs)
        penalty_bps = (best_g - worst_g) * 1e4
        wealth_mult = np.exp((best_g - worst_g) * 50)

        cells = []
        colors = []
        for g in row_grs:
            if np.isnan(g):
                cells.append('N/A')
                colors.append('#F5F5F5')
            elif abs(g - best_g) < 1e-8:
                cells.append(f'{g:.5f} *')
                colors.append('#D4EDDA')  # green highlight
            else:
                cells.append(f'{g:.5f}')
                colors.append('white')

        cells.append(f'{penalty_bps:.0f} bps')
        colors.append('#FFF3CD' if penalty_bps > 20 else 'white')
        cells.append(f'{wealth_mult:.2f}x')
        colors.append('#FFF3CD' if wealth_mult > 1.1 else 'white')
        table_data.append(cells)
        cell_colors.append(colors)

col_labels = [DED_LABELS[d] for d in deds] + ['Max Penalty', '50yr Wealth\nMultiple']

table = ax.table(
    cellText=table_data,
    rowLabels=row_labels,
    colLabels=col_labels,
    cellColours=cell_colors,
    loc='center',
    cellLoc='center',
)
table.auto_set_font_size(False)
table.set_fontsize(8)
table.scale(1, 1.5)

# Bold header row
for (row, col), cell in table.get_celld().items():
    if row == 0:
        cell.set_text_props(fontweight='bold', fontsize=8)
    if col == -1:
        cell.set_text_props(fontsize=8)

ax.set_title(
    'Cost of Getting the Deductible Wrong (* = optimal)',
    fontsize=13, fontweight='bold', pad=20,
)

fig.tight_layout()
save(fig, 'cost_of_getting_it_wrong')
plt.show()

## 11. Breakeven Time
Year at which the median insured path first exceeds the median uninsured path.
Under the exponential growth model, breakeven is immediate if median growth rate
is higher for insured; we report the growth rate differential instead.

In [ ]:
# Breakeven table: growth rate and median growth advantage
print('Breakeven Analysis: Median Growth Rate Comparison')
print('=' * 80)
print(f'{"Config":<30} {"Insured":>10} {"Uninsured":>10} {"Diff (bps)":>12} {"Status":>15}')
print('-' * 80)

breakeven_data = []
for cap in caps:
    for atr in atrs:
        k_ni = find_key(cap, atr, noins=True)
        if k_ni is None: continue
        g_ni_med = np.median(sim[k_ni]['gr'])
        for ded in deds:
            k_ins = find_key(cap, atr, ded=ded)
            if k_ins is None: continue
            g_ins_med = np.median(sim[k_ins]['gr'])
            diff_bps = (g_ins_med - g_ni_med) * 1e4
            status = 'Insured wins' if g_ins_med > g_ni_med else 'Uninsured wins'
            label = f'{fmt(cap)}, ATR={atr}, Ded={DED_LABELS[ded]}'
            print(f'{label:<30} {g_ins_med:>10.5f} {g_ni_med:>10.5f} {diff_bps:>10.1f}   {status:>15}')
            breakeven_data.append({
                'cap': cap, 'atr': atr, 'ded': ded,
                'g_ins': g_ins_med, 'g_ni': g_ni_med,
                'diff_bps': diff_bps,
            })

# Visualize as heatmap
fig, ax = plt.subplots(figsize=(5.5, 4.0))
# Pivot for $5M Cap
cap_focus = 5_000_000
sub = [d for d in breakeven_data if d['cap'] == cap_focus]
if sub:
    atrs_sub = sorted(set(d['atr'] for d in sub))
    deds_sub = sorted(set(d['ded'] for d in sub))
    matrix = np.full((len(deds_sub), len(atrs_sub)), np.nan)
    for d in sub:
        i = deds_sub.index(d['ded'])
        j = atrs_sub.index(d['atr'])
        matrix[i, j] = d['diff_bps']

    im = ax.imshow(matrix, cmap='RdYlGn', aspect='auto',
                   vmin=-max(abs(np.nanmin(matrix)), abs(np.nanmax(matrix))),
                   vmax=max(abs(np.nanmin(matrix)), abs(np.nanmax(matrix))))
    for i in range(len(deds_sub)):
        for j in range(len(atrs_sub)):
            if not np.isnan(matrix[i, j]):
                ax.text(j, i, f'{matrix[i,j]:.1f}', ha='center', va='center', fontsize=9)
    ax.set_xticks(range(len(atrs_sub)))
    ax.set_xticklabels([str(a) for a in atrs_sub])
    ax.set_yticks(range(len(deds_sub)))
    ax.set_yticklabels([DED_LABELS[d] for d in deds_sub])
    ax.set_xlabel('ATR')
    ax.set_ylabel('Deductible')
    plt.colorbar(im, ax=ax, label='Median Growth Advantage (bps)')

ax.set_title(
    f'Median Growth Rate: Insured vs Uninsured (Cap={fmt(cap_focus)})',
    fontsize=12, fontweight='bold',
)
fig.tight_layout()
save(fig, 'breakeven_growth_advantage')
plt.show()

## Summary
All publication figures saved to `output/publication/` at 300 DPI.

**GC Superiority Narrative (5 charts):**
1. Ensemble vs Time Average -- the ergodic gap flagship
2. Ranking Reversal -- slope chart + waterfall decomposition
3. Volatility Tax -- stochastic penalty by configuration
4. Sensitivity to Premium Loading -- crossover at 212% loading
5. Retained Loss Tail Risk -- box plots + exceedance curves

**Supporting Analysis (9 charts):**
6. Optimal Deductible Heatmap
7. Wealth Fan Chart
8. Survival Curves
9. Life/Death Attribution
10. Year-by-Year Growth Lift
11. Peer Benchmark
12. Insurance Outcome Distribution ($0 Ded vs No Insurance)
13. Cost of Getting It Wrong
14. Insurance Lift by Capitalization

In [ ]:
# List all generated figures
figs = sorted(OUTPUT.glob('*.png'))
print(f'Generated {len(figs)} publication figures:')
for f in figs:
    size_kb = f.stat().st_size / 1024
    print(f'  {f.name:<45} {size_kb:>7.0f} KB')

## 10. Insurance Lift Increases with Scale
Growth lift rises as capitalization increases; higher ATR (revenue intensity) amplifies the effect.

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import matplotlib.colors as mcolors
from matplotlib.gridspec import GridSpec
import numpy as np
import pickle
from pathlib import Path

# === User-configurable target ===
TARGET_CAP = 5_000_000
TARGET_ATR = 1.0

# Publication rcParams (see VISUAL_SPECIFICATION.md)
plt.rcParams.update({
    'font.family': 'serif',
    'font.serif': ['CMU Serif', 'Computer Modern', 'Times New Roman', 'DejaVu Serif'],
    'font.size': 10, 'axes.titlesize': 11, 'axes.labelsize': 10,
    'xtick.labelsize': 9, 'ytick.labelsize': 9, 'legend.fontsize': 9,
    'legend.framealpha': 0.9, 'legend.edgecolor': '#CCCCCC',
    'figure.dpi': 150, 'savefig.dpi': 300,
    'savefig.bbox': 'tight', 'savefig.pad_inches': 0.1,
    'axes.spines.top': False, 'axes.spines.right': False,
    'axes.grid': True, 'axes.axisbelow': True,
    'grid.alpha': 0.3, 'grid.linestyle': '--', 'grid.linewidth': 0.5,
    'text.usetex': False, 'mathtext.fontset': 'cm',
})

PAL = {
    'blue': '#0072B2', 'orange': '#E69F00', 'green': '#009E73',
    'vermillion': '#D55E00', 'purple': '#CC79A7', 'sky': '#56B4E9',
    'yellow': '#F0E442', 'gray': '#999999', 'dark': '#333333',
}
DED_COLORS = {0: PAL['blue'], 100_000: PAL['sky'], 250_000: PAL['orange'], 500_000: PAL['vermillion']}
DED_LABELS = {0: '$0', 100_000: '$100K', 250_000: '$250K', 500_000: '$500K'}
DED_COLORS_EXT = {**DED_COLORS, 'noins': PAL['gray']}
DED_LABELS_EXT = {**DED_LABELS, 'noins': 'No Insurance'}
LW_EM, LW_STD, LW_REF, LW_THIN = 2.5, 1.5, 1.0, 0.3

OUTPUT = Path('output/publication')
OUTPUT.mkdir(parents=True, exist_ok=True)

def fmt(n):
    if abs(n) >= 1e9: return f'${n/1e9:.0f}B'
    if abs(n) >= 1e6: return f'${n/1e6:.0f}M'
    if abs(n) >= 1e3: return f'${n/1e3:.0f}K'
    return f'${n:.0f}'

def save(fig, name):
    p = OUTPUT / f'{name}.png'
    fig.savefig(p, dpi=300, facecolor='white')
    print(f'Saved: {p}')

# DECOMP and PRICING are built dynamically in the cache-loading cell below
print("Cell 22 setup complete.")

In [ ]:
import gc

with open('cache/dashboard_cache.pkl', 'rb') as f:
    cache = pickle.load(f)
configs = cache['configs']
crn_pairs = cache['crn_pairs']
param_values = cache['param_values']
RESULTS_DIR = Path(cache['results_dir'])

def find_key(cap, atr, ded=None, noins=False):
    for k, c in configs.items():
        if c['Cap'] == cap and c['ATR'] == atr:
            if noins and c.get('NOINS'): return k
            if not noins and not c.get('NOINS') and c.get('Ded') == ded: return k
    return None

def ruin_val(cfg, yr):
    rp = cfg['ruin_probability']
    for k, v in rp.items():
        if int(k) == yr: return v
    return None

def get_per_sim_ruin(key, min_trailing=2):
    path = RESULTS_DIR / f'{key}.pkl'
    with open(path, 'rb') as f:
        data = pickle.load(f)
    losses = data.annual_losses
    n_sims, n_years = losses.shape
    last_active = np.full(n_sims, -1, dtype=int)
    for yr in range(n_years - 1, -1, -1):
        active = (losses[:, yr] != 0) & (last_active == -1)
        last_active[active] = yr
    ruined = last_active < (n_years - min_trailing)
    del data
    gc.collect()
    return ruined

sim = {}
def load_sim(key):
    if key in sim: return sim[key]
    path = RESULTS_DIR / f'{key}.pkl'
    with open(path, 'rb') as f:
        data = pickle.load(f)
    sim[key] = {'gr': data.growth_rates.copy(), 'fa': data.final_assets.copy()}
    del data
    return sim[key]

print('Loading simulation arrays...')
for cap in param_values['Cap']:
    for atr in param_values['ATR']:
        k = find_key(cap, atr, noins=True)
        if k: load_sim(k)
        for ded in param_values['Ded']:
            k = find_key(cap, atr, ded=ded)
            if k: load_sim(k)
deds = param_values['Ded']
caps = param_values['Cap']
atrs = param_values['ATR']
print(f'Loaded {len(sim)} configs')

# ---------------------------------------------------------------------------
# Dynamic DECOMP / PRICING builder
# ---------------------------------------------------------------------------
def build_decomp(cap, atr, ebitabl=0.125, lr=0.70, tax_rate=0.25,
                 retention_ratio=0.70, pricing_iters=250_000, seed=42):
    """Build DECOMP and PRICING dicts for any (Cap, ATR) pair."""
    from ergodic_insurance.loss_distributions import ManufacturingLossGenerator

    revenue = cap * atr
    DET_RATIO = 0.556  # g_det/g_naive, empirically stable across all configs

    generator = ManufacturingLossGenerator(
        attritional_params={
            "base_frequency": 2.85 * revenue / 10_000_000,
            "severity_mean": 40_000,
            "severity_cv": 0.8,
            "revenue_scaling_exponent": 1.0,
            "reference_revenue": revenue,
        },
        large_params={
            "base_frequency": 0.20 * revenue / 10_000_000,
            "severity_mean": 500_000,
            "severity_cv": 1.5,
            "revenue_scaling_exponent": 1.0,
            "reference_revenue": revenue,
        },
        catastrophic_params={
            "base_frequency": 0.02 * revenue / 10_000_000,
            "severity_xm": 5_000_000,
            "severity_alpha": 2.5,
            "revenue_scaling_exponent": 1.0,
            "reference_revenue": revenue,
        },
        seed=seed,
    )

    # Run pricing sim for each deductible
    deductibles = {'$0K': 0, '$100K': 100_000, '$250K': 250_000, '$500K': 500_000}
    policy_limit = 100_000_000_000
    expected_insured, expected_retained = {}, {}
    for label, ded in deductibles.items():
        total_insured = 0.0
        total_retained = 0.0
        for _ in range(pricing_iters):
            loss_events, _ = generator.generate_losses(duration=1, revenue=revenue)
            for ev in loss_events:
                ins = max(min(ev.amount, policy_limit) - ded, 0)
                total_insured += ins
                total_retained += ev.amount - ins
        expected_insured[label] = total_insured / pricing_iters
        expected_retained[label] = total_retained / pricing_iters

    # Build PRICING dict
    pricing = {
        'EBITABL': ebitabl, 'tax_rate': tax_rate,
        'retention_ratio': retention_ratio, 'base_LR': lr,
        'expected_losses': expected_insured,
        'expected_retained': expected_retained,
        'revenue': revenue,
    }

    # Build DECOMP for each deductible + No Insurance
    decomp = {}
    for label in list(deductibles) + ['No Ins']:
        # g_actual from dashboard cache
        if label == 'No Ins':
            k = find_key(cap, atr, noins=True)
        else:
            k = find_key(cap, atr, ded=deductibles[label])
        g_actual = configs[k]['growth_rate_mean'] * 10000

        # g_naive from analytical formula
        if label == 'No Ins':
            cost_rate = expected_insured['$0K'] / revenue  # full ground-up, no premium
        else:
            premium = expected_insured[label] / lr
            cost_rate = (premium + expected_retained[label]) / revenue
        g_naive = (ebitabl - cost_rate) * (1 - tax_rate) * retention_ratio * 10000

        g_det = g_naive * DET_RATIO
        decomp[label] = {
            'g_naive': round(g_naive, 1), 'g_det': round(g_det, 1),
            'g_actual': round(g_actual, 1),
            'tax_drag': round(g_naive - g_det, 1),
            'stoch_pen': round(g_det - g_actual, 1),
        }

    return decomp, pricing

# Build DECOMP for selected configuration
DECOMP, PRICING = build_decomp(TARGET_CAP, TARGET_ATR)
print(f'DECOMP for Cap=${TARGET_CAP/1e6:.0f}M, ATR={TARGET_ATR}:')
for label, d in DECOMP.items():
    print(f'  {label:>8}: g_naive={d["g_naive"]:6.1f}  g_det={d["g_det"]:6.1f}  '
          f'g_actual={d["g_actual"]:6.1f}  stoch_pen={d["stoch_pen"]:+6.1f}')

In [ ]:
P = PRICING
tax_ratio = 0.556  # DET_RATIO, consistent with build_decomp
lr_range = np.linspace(0.15, 1.0, 200)

def g_actual_fn(label, lr):
    e_ins = P['expected_losses'][label]
    e_ret = P['expected_retained'][label]
    sp = DECOMP[label]['stoch_pen']
    g_naive = (P['EBITABL'] - (e_ins / lr + e_ret) / P['revenue']) \
              * (1 - P['tax_rate']) * P['retention_ratio'] * 10000
    return g_naive * tax_ratio - sp

fig, ax = plt.subplots(figsize=(5.5, 4.0))
ax.axvspan(0.50, 0.80, color=PAL['green'], alpha=0.08, label='Realistic pricing zone')

ded_configs = [
    ('$0K', PAL['blue'], '-', LW_EM),
    ('$100K', PAL['sky'], '-', LW_STD),
    ('$250K', PAL['orange'], '-', LW_STD),
    ('$500K', PAL['vermillion'], '-', LW_STD),
]

for label, color, ls, lw in ded_configs:
    g_vals = np.array([g_actual_fn(label, lr) for lr in lr_range])
    ax.plot(lr_range, g_vals, color=color, lw=lw, ls=ls, label=f'Ded {label}')
    g_sim = DECOMP[label]['g_actual']
    ax.plot(0.70, g_sim, marker='D', ms=7, color=color, markeredgecolor='white',
            markeredgewidth=0.5, zorder=5)

g_noins = DECOMP['No Ins']['g_actual']
ax.axhline(g_noins, color=PAL['gray'], lw=LW_STD, ls='--', label='No Insurance')

# Find crossover: iterate from HIGH LR down to find where $500K first beats $0K
for lr_test in reversed(lr_range):
    g0 = g_actual_fn('$0K', lr_test)
    g5 = g_actual_fn('$500K', lr_test)
    if g5 >= g0:
        crossover_lr = lr_test
        crossover_g = g0
        loading_pct = (1/lr_test - 1) * 100
        ax.plot(crossover_lr, crossover_g, 'X', ms=10, color=PAL['dark'], zorder=6)
        ax.annotate(
            f'Crossover at LR={crossover_lr:.2f}\n({loading_pct:.0f}% loading)',
            xy=(crossover_lr, crossover_g),
            xytext=(crossover_lr + 0.15, crossover_g + 40),
            fontsize=8,
            arrowprops=dict(arrowstyle='->', color=PAL['dark'], lw=0.8),
            bbox=dict(boxstyle='round,pad=0.3', fc=PAL['yellow'], alpha=0.7,
                      ec=PAL['dark'], lw=0.5),
        )
        print(f'Crossover: LR={crossover_lr:.3f}, loading={loading_pct:.0f}%')
        break

ax.axvline(0.70, color=PAL['gray'], ls=':', lw=0.8, alpha=0.5)
ax.text(0.71, ax.get_ylim()[0] + 5, 'Simulated\nLR=0.70', fontsize=7,
        color=PAL['gray'], va='bottom')

ax.set_xlabel('Loss Ratio (lower = more expensive insurance)')
ax.set_ylabel('Time-Average Growth Rate (bps/yr)')
ax.set_title('How Expensive Must Insurance Be Before Retention Wins?',
             fontsize=12, fontweight='bold')
ax.legend(loc='upper right', fontsize=7.5, ncol=2)

fig.tight_layout()
save(fig, 'sensitivity_premium_loading')
plt.show()

In [ ]:
# === Retained Loss Tail Risk ===
N_YEARS_SIM = 100_000
A0 = TARGET_CAP
revenue = TARGET_CAP * TARGET_ATR
rng_loss = np.random.default_rng(42)

# Loss distribution params (scaled by revenue)
att_freq = 2.85 * revenue / 10_000_000
att_cv = 0.8
att_sigma = np.sqrt(np.log(1 + att_cv**2))
att_mu = np.log(40_000) - att_sigma**2 / 2

lg_freq = 0.20 * revenue / 10_000_000
lg_cv = 1.5
lg_sigma = np.sqrt(np.log(1 + lg_cv**2))
lg_mu = np.log(500_000) - lg_sigma**2 / 2

cat_freq = 0.02 * revenue / 10_000_000
cat_xm = 5_000_000
cat_alpha = 2.5

deductibles_sim = {'\\$0K': 0, '\\$100K': 100_000, '\\$250K': 250_000,
               '\\$500K': 500_000, 'No Ins': np.inf}
retained_annual = {k: np.zeros(N_YEARS_SIM) for k in deductibles_sim}

for yr in range(N_YEARS_SIM):
    year_losses = []
    n_att = rng_loss.poisson(att_freq)
    if n_att > 0:
        year_losses.extend(rng_loss.lognormal(att_mu, att_sigma, n_att).tolist())
    n_lg = rng_loss.poisson(lg_freq)
    if n_lg > 0:
        year_losses.extend(rng_loss.lognormal(lg_mu, lg_sigma, n_lg).tolist())
    n_cat = rng_loss.poisson(cat_freq)
    if n_cat > 0:
        u = rng_loss.uniform(0, 1, n_cat)
        cat_losses = cat_xm * u ** (-1 / cat_alpha)
        year_losses.extend(cat_losses.tolist())
    for label, ded in deductibles_sim.items():
        total_retained = 0.0
        for loss in year_losses:
            if ded == np.inf:
                total_retained += loss
            else:
                total_retained += min(loss, ded)
        retained_annual[label][yr] = total_retained

retained_pct = {k: v / A0 * 100 for k, v in retained_annual.items()}
print("Annual retained losses (% of assets):")
for label in deductibles_sim:
    vals = retained_pct[label]
    print(f"  {label:>8}: mean={vals.mean():.2f}%, std={vals.std():.2f}%, "
          f"max={vals.max():.1f}%, P99={np.percentile(vals, 99):.2f}%")

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(7.0, 3.5))

ded_keys_box = ['\\$100K', '\\$250K', '\\$500K', 'No Ins']
ded_colors_box = [PAL['sky'], PAL['orange'], PAL['vermillion'], PAL['gray']]
data_box = [retained_pct[k] for k in ded_keys_box]

bp = ax1.boxplot(data_box, tick_labels=ded_keys_box, patch_artist=True, showfliers=False,
                 whiskerprops=dict(lw=0.8), capprops=dict(lw=0.8),
                 medianprops=dict(color=PAL['dark'], lw=1.5))
for patch, color in zip(bp['boxes'], ded_colors_box):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)

for i, k in enumerate(ded_keys_box):
    p99 = np.percentile(retained_pct[k], 99)
    p999 = np.percentile(retained_pct[k], 99.9)
    if p99 > 0:
        ax1.scatter(i + 1, p99, marker='v', s=25, color=ded_colors_box[i],
                    zorder=5, label='P99' if i == 0 else None)
    if p999 > 0 and p999 > p99 * 1.2:
        ax1.scatter(i + 1, p999, marker='^', s=25, color=PAL['vermillion'],
                    zorder=5, label='P99.9' if i == 0 else None)

ax1.set_yscale('log')
ax1.set_ylim(0.01, None)
ax1.set_ylabel('Annual Retained Loss (% of assets)')
ax1.set_title('A. Retained Loss Distribution', fontsize=11, fontweight='bold')

ax1.text(0.03, 0.03, '\\$0K Ded: zero retained losses\n(all risk transferred)',
         transform=ax1.transAxes, fontsize=7, va='bottom',
         bbox=dict(boxstyle='round,pad=0.3', fc='#D4EDDA', alpha=0.8,
                   ec=PAL['green'], lw=0.5))

noins_max = retained_pct['No Ins'].max()
ax1.annotate(f'Max: {noins_max:.0f}%', xy=(4, noins_max),
             xytext=(3.0, noins_max * 0.25),
             fontsize=7, arrowprops=dict(arrowstyle='->', lw=0.6, color=PAL['dark']),
             bbox=dict(boxstyle='round,pad=0.2', fc='white', ec=PAL['gray'], lw=0.5))
ax1.legend(fontsize=7, loc='upper left')

# Panel B
ded_keys_all = ['\\$0K', '\\$100K', '\\$250K', '\\$500K', 'No Ins']
ded_colors_all = [PAL['blue'], PAL['sky'], PAL['orange'], PAL['vermillion'], PAL['gray']]
thresholds = np.logspace(-1, 4, 500)

for i, k in enumerate(ded_keys_all):
    if k == '\\$0K': continue
    vals = retained_pct[k]
    exceed = np.array([np.mean(vals > t) for t in thresholds])
    valid = exceed > 0
    if valid.sum() > 0:
        ax2.plot(thresholds[valid], exceed[valid], color=ded_colors_all[i],
                 lw=LW_STD, label=k)

ax2.axvline(100, color=PAL['vermillion'], ls=':', lw=LW_REF, alpha=0.7)
ax2.text(120, 0.15, '100% of assets\n(existential)', fontsize=7,
         color=PAL['vermillion'], va='top', rotation=90)

ax2.set_xscale('log')
ax2.set_yscale('log')
ax2.set_xlabel('Loss Threshold (% of assets)')
ax2.set_ylabel('Exceedance Probability')
ax2.set_title('B. Tail Risk Exceedance', fontsize=11, fontweight='bold')
ax2.legend(fontsize=7.5, loc='upper right')

fig.suptitle('Retained Loss Tail Risk Explodes with Deductible',
             fontsize=13, fontweight='bold', y=1.04)
fig.tight_layout()
save(fig, 'retained_loss_tail_risk')
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(7.0, 3.0), sharey=True)

for ax_idx, atr in enumerate(atrs):
    ax = axes[ax_idx]
    for ded in deds:
        cap_vals = []
        lift_vals = []
        for cap in caps:
            k_ins = find_key(cap, atr, ded=ded)
            k_ni = find_key(cap, atr, noins=True)
            if k_ins and k_ni:
                g_ins = configs[k_ins]['growth_rate_mean']
                g_ni = configs[k_ni]['growth_rate_mean']
                cap_vals.append(cap / 1e6)
                lift_vals.append((g_ins - g_ni) * 1e4)  # basis points
        if cap_vals:
            ax.plot(cap_vals, lift_vals, color=DED_COLORS[ded], lw=LW_EM,
                    marker='o', ms=5, label=f'Ded {DED_LABELS[ded]}')

    ax.axhline(0, color=PAL['gray'], ls=':', lw=LW_REF)
    ax.set_title(f'ATR = {atr}', fontsize=10)
    ax.set_xlabel('Capitalization ($M)')
    if ax_idx == 0:
        ax.set_ylabel('Growth Lift (basis points)')
    if ax_idx == 2:
        ax.legend(fontsize=7, loc='upper right')

fig.suptitle(
    'Insurance Lift Increases with Capitalization',
    fontsize=13, fontweight='bold', y=1.04,
)
fig.tight_layout()
save(fig, 'insurance_value_lift_with_scale')
plt.show()

In [ ]:

# === The Ranking Reversal: Slope Chart (single panel) ===
fig, ax = plt.subplots(figsize=(5.5, 4.0))

configs_list = ['$0K', '$100K', '$250K', '$500K', 'No Ins']
colors_slope = [PAL['blue'], PAL['sky'], PAL['orange'], PAL['vermillion'], PAL['gray']]
g_det = np.array([DECOMP[c]['g_det'] for c in configs_list])
g_actual = np.array([DECOMP[c]['g_actual'] for c in configs_list])

# Rank: 1 = best (highest growth rate)
det_rank = len(g_det) - np.argsort(np.argsort(g_det))
actual_rank = len(g_actual) - np.argsort(np.argsort(g_actual))

for i, (label, color) in enumerate(zip(configs_list, colors_slope)):
    ax.plot([0, 1], [det_rank[i], actual_rank[i]], color=color,
            lw=LW_EM, marker='o', ms=9, markerfacecolor=color,
            markeredgecolor='white', markeredgewidth=0.8, zorder=3)
    # Left labels (deterministic ranking)
    ax.text(-0.06, det_rank[i], f'{label}\n{g_det[i]:.0f} bps',
            ha='right', va='center', fontsize=9, color=color, fontweight='bold')
    # Right labels (actual ranking)
    ax.text(1.06, actual_rank[i], f'{label}\n{g_actual[i]:.0f} bps',
            ha='left', va='center', fontsize=9, color=color, fontweight='bold')

ax.set_xlim(-0.45, 1.45)
ax.set_ylim(0.3, 5.7)
ax.invert_yaxis()
ax.set_xticks([0, 1])
ax.set_xticklabels(['Deterministic\n(expected-value)', 'Actual\n(time-average)'], fontsize=10)
ax.set_yticks([])
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.grid(False)

ax.set_title('The Ranking Reversal:\nExpected Value Gets It Backwards',
             fontsize=13, fontweight='bold', pad=12)

fig.tight_layout()
save(fig, 'ranking_reversal_slope')
plt.show()


In [ ]:
DECOMP, PRICING = build_decomp(5_000_000, 1.0)

# === The Volatility Tax vs Premium Savings: Grouped Bar Chart ===
from matplotlib.patches import Patch

configs_labels = ['$0K', '$100K', '$250K', '$500K', 'No Ins']
x_labels = ['\\$0K\n(GC)', '\\$100K', '\\$250K', '\\$500K', 'No\nInsurance']
n_configs = len(configs_labels)

# Extract data from DECOMP
g_det = [DECOMP[c]['g_det'] for c in configs_labels]
g_actual = [DECOMP[c]['g_actual'] for c in configs_labels]
vol_impact = [-(DECOMP[c]['stoch_pen']) for c in configs_labels]  # Negative = tax

fig, ax = plt.subplots(figsize=(7.0, 4.5))

x = np.arange(n_configs)
w = 0.22

bars_det = ax.bar(x - w, g_det, w, color=PAL['green'], edgecolor='white', lw=0.5, zorder=3)
bar_colors_vol = [PAL['green'] if vi > 0 else PAL['vermillion'] for vi in vol_impact]
bars_vol = ax.bar(x, vol_impact, w, color=PAL['vermillion'], edgecolor='white', lw=0.5, zorder=3)
bars_net = ax.bar(x + w, g_actual, w, color=PAL['blue'], edgecolor='white', lw=0.5, zorder=3)

for i in range(n_configs):
    # Deterministic: label inside top of bar (white text)
    ax.text(x[i] - w, g_det[i] - 8, f'{g_det[i]:.0f}', ha='center', va='top',
            fontsize=7.5, color='white', fontweight='bold')
    # Volatility impact: label outside bar
    va = 'bottom' if vol_impact[i] > 0 else 'top'
    offset = 4 if vol_impact[i] > 0 else -4
    ax.text(x[i], vol_impact[i] + offset, f'{vol_impact[i]:+.0f}', ha='center', va=va,
            fontsize=7, color=bar_colors_vol[i], fontweight='bold')
    # Net: label inside top of bar (white text)
    ax.text(x[i] + w, g_actual[i] - 8, f'{g_actual[i]:.0f}', ha='center', va='top',
            fontsize=7.5, color='white', fontweight='bold')

ax.axhline(0, color=PAL['dark'], lw=0.8)

ax.set_xticks(x)
ax.set_xticklabels(x_labels, fontsize=9)
ax.set_ylabel('Growth Rate Impact (bps/yr)')
ax.set_title('The Volatility Tax Overwhelms Premium Savings',
             fontsize=13, fontweight='bold', pad=15)
ax.set_ylim(-165, 280)

# Annotation
# ax.annotate(
#     'Insurance reduces\nvolatility drag',
#     xy=(0, vol_impact[0]),
#     xytext=(0.22, 0.32),
#     textcoords='axes fraction',
#     fontsize=8,
#     arrowprops=dict(arrowstyle='->', color=PAL['green'], lw=0.8),
#     bbox=dict(boxstyle='round,pad=0.3', fc='#D4EDDA', alpha=0.8,
#               ec=PAL['green'], lw=0.5))

legend_elements = [
    Patch(facecolor=PAL['green'], edgecolor='white', label='Expected-Value Growth (no volatility)'),
    Patch(facecolor=PAL['vermillion'], edgecolor='white', label='Volatility Tax'),
    Patch(facecolor=PAL['blue'], edgecolor='white', label='Actual Growth (net)'),
]
ax.legend(handles=legend_elements, loc='best', fontsize=8)

fig.tight_layout()
fig.savefig(fr'{OUTPUT}\volatility_tax_vs_premium_savings', dpi=300, facecolor='white')
plt.show()


In [ ]:

from pathlib import Path
figs = sorted(Path('output/publication').glob('*.png'))
print(f'Generated {len(figs)} publication figures:')
for f in figs:
    size_kb = f.stat().st_size / 1024
    print(f'  {f.name:<50} {size_kb:>7.0f} KB')


In [ ]:

import pickle
import numpy as np

# Load the dashboard cache
with open(r"C:\Users\alexf\OneDrive\Documents\Projects\Ergodic Insurance Limits\ergodic_insurance\notebooks\results_vol_sim\cache\dashboard_cache.pkl", "rb") as f:
    dc = pickle.load(f)

# Find the $10M, ATR=1.0, $100K deductible config
target = None
target_key = None
for k, v in dc["configs"].items():
    if (v["Cap"] == 10_000_000 and v["ATR"] == 1.0 and v.get("Ded") == 100_000 and not v.get("NOINS")):
        target = v
        target_key = k
        break

# Also find the no-insurance pair
noins = None
noins_key = dc.get("crn_pairs", {}).get(target_key)
if noins_key:
    noins = dc["configs"][noins_key]

print("=" * 70)
print("TARGET CONFIG")
print("=" * 70)
print(f"Key: {target_key}")
print(f"Cap: ${target['Cap']:,.0f}")
print(f"ATR: {target['ATR']}")
print(f"Ded: ${target['Ded']:,.0f}")
print(f"EBITABL: {target['EBITABL']}")
print(f"LR: {target['LR']}")
print(f"Vol: {target['Vol']}")
print(f"n_sims: {target['n_sims']:,}")
print(f"n_years: {target['n_years']}")
print()
print(f"Growth rate mean: {target['growth_rate_mean']:.6f} ({target['growth_rate_mean']*10000:.1f} bps/yr)")
print(f"Growth rate median: {target['growth_rate_median']:.6f} ({target['growth_rate_median']*10000:.1f} bps/yr)")
print(f"Growth rate std: {target['growth_rate_std']:.6f}")
print()
print(f"Final assets mean: ${target['final_assets_mean']:,.0f}")
print(f"Final assets median: ${target['final_assets_median']:,.0f}")
print()
print("Ruin probability:")
for yr, prob in sorted(target['ruin_probability'].items()):
    print(f"  Year {yr}: {prob:.4%}")

if noins:
    print()
    print("=" * 70)
    print("NO INSURANCE PAIR")
    print("=" * 70)
    print(f"Key: {noins_key}")
    print(f"Growth rate mean: {noins['growth_rate_mean']:.6f} ({noins['growth_rate_mean']*10000:.1f} bps/yr)")
    print(f"Growth rate median: {noins['growth_rate_median']:.6f} ({noins['growth_rate_median']*10000:.1f} bps/yr)")
    print(f"Final assets mean: ${noins['final_assets_mean']:,.0f}")
    print(f"Final assets median: ${noins['final_assets_median']:,.0f}")
    print()
    print("Ruin probability:")
    for yr, prob in sorted(noins['ruin_probability'].items()):
        print(f"  Year {yr}: {prob:.4%}")


In [ ]:

# Now let's compute the EBITDA analytically, step by step
# We need the actual pricing data. Let me replicate the pricing simulation.

print("=" * 70)
print("STEP 1: MODEL PARAMETERS")
print("=" * 70)

INITIAL_ASSETS = 10_000_000
ATR = 1.0
EBITABL = 0.125      # Operating margin before insurance and losses
TAX_RATE = 0.25
RETENTION_RATIO = 0.70  # 70% retained, 30% dividends
PPE_RATIO = 0.00        # No depreciation
DEDUCTIBLE = 100_000
LOSS_RATIO = 0.70       # Insurance pricing: Premium = Expected Insured Loss / 0.70
REVENUE_VOL = 0.15      # GBM sigma

REVENUE = INITIAL_ASSETS * ATR

print(f"Initial Assets:     ${INITIAL_ASSETS:>12,.0f}")
print(f"Asset Turnover:     {ATR:>12.1f}")
print(f"Revenue (Yr 1):     ${REVENUE:>12,.0f}")
print(f"EBITABL Margin:     {EBITABL:>12.1%}")
print(f"Tax Rate:           {TAX_RATE:>12.1%}")
print(f"Retention Ratio:    {RETENTION_RATIO:>12.1%}")
print(f"Deductible:         ${DEDUCTIBLE:>12,.0f}")
print(f"Loss Ratio (LR):    {LOSS_RATIO:>12.2f}")
print(f"Revenue Vol (σ):    {REVENUE_VOL:>12.2f}")


In [ ]:

# STEP 2: Estimate expected losses using the simulation's loss distribution parameters
# We'll run a quick pricing simulation identical to what run_vol_sim_colab.py does

import numpy as np
import sys
sys.path.insert(0, r"C:\Users\alexf\OneDrive\Documents\Projects\Ergodic Insurance Limits")

from ergodic_insurance.loss_distributions import ManufacturingLossGenerator

print("=" * 70)
print("STEP 2: LOSS DISTRIBUTION PARAMETERS")
print("=" * 70)

cur_revenue = float(REVENUE)

# These are the exact parameters from run_vol_sim_colab.py
attritional_params = {
    "base_frequency": 2.85 * cur_revenue / 10_000_000,
    "severity_mean": 40_000,
    "severity_cv": 0.8,
    "revenue_scaling_exponent": 1.0,
    "reference_revenue": cur_revenue,
}
large_params = {
    "base_frequency": 0.20 * cur_revenue / 10_000_000,
    "severity_mean": 500_000,
    "severity_cv": 1.5,
    "revenue_scaling_exponent": 1.0,
    "reference_revenue": cur_revenue,
}
catastrophic_params = {
    "base_frequency": 0.02 * cur_revenue / 10_000_000,
    "severity_xm": 5_000_000,
    "severity_alpha": 2.5,
    "revenue_scaling_exponent": 1.0,
    "reference_revenue": cur_revenue,
}

print(f"Attritional: freq={attritional_params['base_frequency']:.2f}/yr, "
      f"mean=${attritional_params['severity_mean']:,.0f}, CV={attritional_params['severity_cv']}")
print(f"Large:       freq={large_params['base_frequency']:.2f}/yr, "
      f"mean=${large_params['severity_mean']:,.0f}, CV={large_params['severity_cv']}")
print(f"Catastrophic: freq={catastrophic_params['base_frequency']:.3f}/yr, "
      f"xm=${catastrophic_params['severity_xm']:,.0f}, α={catastrophic_params['severity_alpha']}")

# Expected losses analytically
# Attritional: Poisson(2.85) x LogNormal(mean=40K, CV=0.8)
att_freq = attritional_params['base_frequency']
att_mean_sev = attritional_params['severity_mean']
att_expected = att_freq * att_mean_sev

# Large: Poisson(0.20) x LogNormal(mean=500K, CV=1.5)
lg_freq = large_params['base_frequency']
lg_mean_sev = large_params['severity_mean']
lg_expected = lg_freq * lg_mean_sev

# Cat: Poisson(0.02) x Pareto(xm=5M, alpha=2.5)
cat_freq = catastrophic_params['base_frequency']
cat_alpha = catastrophic_params['severity_alpha']
cat_xm = catastrophic_params['severity_xm']
cat_mean_sev = cat_xm * cat_alpha / (cat_alpha - 1)  # Pareto mean
cat_expected = cat_freq * cat_mean_sev

total_expected_gu = att_expected + lg_expected + cat_expected

print(f"\nExpected Annual Ground-Up Losses (analytical):")
print(f"  Attritional:  {att_freq:.2f} × ${att_mean_sev:>10,.0f} = ${att_expected:>12,.0f}")
print(f"  Large:        {lg_freq:.2f} × ${lg_mean_sev:>10,.0f} = ${lg_expected:>12,.0f}")
print(f"  Catastrophic: {cat_freq:.3f} × ${cat_mean_sev:>10,.0f} = ${cat_expected:>12,.0f}")
print(f"  {'─'*52}")
print(f"  Total Ground-Up:                        ${total_expected_gu:>12,.0f}")
print(f"  As % of Revenue:                        {total_expected_gu/cur_revenue:>12.2%}")
print(f"  As % of Assets:                         {total_expected_gu/INITIAL_ASSETS:>12.2%}")


In [ ]:

# STEP 3: Run the actual pricing simulation (same as the simulation code does)
# This gives us the exact premium and retained loss figures

print("=" * 70)
print("STEP 3: PRICING SIMULATION (100K iterations)")
print("=" * 70)

# Use the same seed logic as run_vol_sim_colab.py
# For $10M, ATR=1.0, $100K ded, we need to find the right index
# The index determines the pricing seed
# Let's just run with a large number of iterations for stable estimates

crn_base_seed = 20260130

# We need to figure out what index was used for this config
# From the simulation code, index is the position in the parameter grid
# But we can just run pricing with enough iterations - the expected values converge

PRICING_SIMULATIONS = 500_000  # Large for stable estimates

generator_pricing = ManufacturingLossGenerator(
    attritional_params=attritional_params,
    large_params=large_params,
    catastrophic_params=catastrophic_params,
    seed=42,  # Any seed works - we just need expected values
)

deductible = DEDUCTIBLE
policy_limit = 100_000_000_000  # Unlimited

total_insured_loss = 0.0
total_retained_loss = 0.0
total_ground_up = 0.0
n_claims = 0

for yr in range(PRICING_SIMULATIONS):
    loss_events, loss_meta = generator_pricing.generate_losses(duration=1, revenue=cur_revenue)
    for loss_event in loss_events:
        n_claims += 1
        gu_loss = loss_event.amount
        insured_loss = max(min(gu_loss, policy_limit) - deductible, 0)
        retained_loss = gu_loss - insured_loss
        
        total_insured_loss += insured_loss
        total_retained_loss += retained_loss
        total_ground_up += gu_loss

avg_annual_insured = total_insured_loss / PRICING_SIMULATIONS
avg_annual_retained = total_retained_loss / PRICING_SIMULATIONS
avg_annual_gu = total_ground_up / PRICING_SIMULATIONS
avg_claims_per_yr = n_claims / PRICING_SIMULATIONS

premium = avg_annual_insured / LOSS_RATIO
total_cost_of_risk = premium + avg_annual_retained

print(f"Pricing iterations: {PRICING_SIMULATIONS:,}")
print(f"Average claims/year: {avg_claims_per_yr:.2f}")
print()
print(f"Per-Occurrence Deductible: ${deductible:>12,.0f}")
print(f"{'─'*52}")
print(f"Avg Annual Ground-Up Loss:    ${avg_annual_gu:>12,.0f}")
print(f"Avg Annual Insured Loss:      ${avg_annual_insured:>12,.0f}")
print(f"Avg Annual Retained Loss:     ${avg_annual_retained:>12,.0f}")
print(f"  (check: insured + retained = ${avg_annual_insured + avg_annual_retained:>10,.0f})")
print(f"{'─'*52}")
print(f"Annual Premium (insured/LR):  ${premium:>12,.0f}")
print(f"  Premium as % of Revenue:    {premium/cur_revenue:>12.2%}")
print(f"{'─'*52}")
print(f"Total Cost of Risk:           ${total_cost_of_risk:>12,.0f}")
print(f"  as % of Revenue:            {total_cost_of_risk/cur_revenue:>12.2%}")
print(f"  as % of Assets:             {total_cost_of_risk/INITIAL_ASSETS:>12.2%}")


In [ ]:

# STEP 4: INCOME STATEMENT - FROM REVENUE TO NET INCOME

print("=" * 70)
print("STEP 4: YEAR 1 INCOME STATEMENT (Deterministic)")
print("=" * 70)
print()
print(f"  Revenue                          ${REVENUE:>12,.0f}")
print(f"  × EBITABL Margin                 {EBITABL:>12.1%}")
print(f"  ─────────────────────────────────────────────")
print(f"  EBITDA (before insurance/losses)  ${REVENUE * EBITABL:>12,.0f}")

ebitda_before = REVENUE * EBITABL

print(f"  − Insurance Premium              ${premium:>12,.0f}")
print(f"  − Expected Retained Losses       ${avg_annual_retained:>12,.0f}")
print(f"  ─────────────────────────────────────────────")

ebitda_after = ebitda_before - premium - avg_annual_retained

print(f"  EBITDA (after insurance/losses)   ${ebitda_after:>12,.0f}")
print(f"  EBITDA margin (after):            {ebitda_after/REVENUE:>12.2%}")
print()

# Since PPE = 0%, there's no depreciation, so EBITDA = EBIT = Operating Income
print(f"  (PPE = 0%, so no depreciation: EBITDA = EBIT)")
print()
print(f"  EBIT                             ${ebitda_after:>12,.0f}")
print(f"  × (1 − Tax Rate)                {1 - TAX_RATE:>12.1%}")
print(f"  ─────────────────────────────────────────────")

net_income = ebitda_after * (1 - TAX_RATE)
print(f"  Net Income                       ${net_income:>12,.0f}")
print()

retained_earnings = net_income * RETENTION_RATIO
dividends = net_income * (1 - RETENTION_RATIO)

print(f"  × Retention Ratio                {RETENTION_RATIO:>12.1%}")
print(f"  ─────────────────────────────────────────────")
print(f"  Retained Earnings                ${retained_earnings:>12,.0f}")
print(f"  Dividends Paid                   ${dividends:>12,.0f}")


In [ ]:

# STEP 5: GROWTH RATE DECOMPOSITION
# Connect the income statement to the observed growth rate

print("=" * 70)
print("STEP 5: GROWTH RATE DECOMPOSITION")
print("=" * 70)

# g_naive: the simple analytical formula (ignores tax accrual timing)
total_cost_rate = total_cost_of_risk / REVENUE
g_naive = (EBITABL - total_cost_rate) * (1 - TAX_RATE) * RETENTION_RATIO

print(f"\n  A. Naive Analytical Growth Rate")
print(f"     g_naive = (EBITABL − cost_rate) × (1−tax) × retention")
print(f"            = ({EBITABL:.3f} − {total_cost_rate:.4f}) × {1-TAX_RATE:.2f} × {RETENTION_RATIO:.2f}")
print(f"            = {EBITABL - total_cost_rate:.4f} × {(1-TAX_RATE)*RETENTION_RATIO:.4f}")
print(f"            = {g_naive:.6f}  ({g_naive*10000:.1f} bps/yr)")
print(f"     = Retained Earnings / Assets = ${retained_earnings:,.0f} / ${INITIAL_ASSETS:,.0f}")
print(f"            = {retained_earnings/INITIAL_ASSETS:.6f}  ({retained_earnings/INITIAL_ASSETS*10000:.1f} bps/yr)")

# Compare with the $5M DECOMP values to estimate tax drag and stochastic penalty
# From DECOMP ($5M, $100K): g_naive=412.0, g_det=229.3, g_actual=175.7
# Tax drag = 412.0 - 229.3 = 182.7 bps
# Stochastic penalty = 229.3 - 175.7 = 53.6 bps

# For $10M, the rates are nearly identical (see GC_SUPERIORITY_EXPLANATION.md section 4.2)
# because loss frequency scales linearly with revenue

g_naive_5m = 0.04120
g_det_5m = 0.02293
g_actual_5m = 0.01757
tax_drag_5m = g_naive_5m - g_det_5m
stoch_pen_5m = g_det_5m - g_actual_5m

print(f"\n  B. Growth Rate Decomposition ($5M reference from DECOMP)")
print(f"     g_naive  = {g_naive_5m*10000:.1f} bps")
print(f"     g_det    = {g_det_5m*10000:.1f} bps  (tax drag = {tax_drag_5m*10000:.1f} bps)")
print(f"     g_actual = {g_actual_5m*10000:.1f} bps  (stoch pen = {stoch_pen_5m*10000:.1f} bps)")

# For $10M, compute the actual decomposition
g_actual_10m = target['growth_rate_mean']
# Estimate g_det assuming same tax drag ratio
tax_drag_ratio = tax_drag_5m / g_naive_5m  # ~44.3% of naive growth
g_det_10m_est = g_naive - (g_naive * tax_drag_ratio)
# But better: the GC_SUPERIORITY doc says growth rates are nearly identical across cap sizes
# So we can use the $5M stochastic penalty directly

stoch_pen_10m = g_det_5m - g_actual_10m  # Approximate (using $5M g_det)
tax_drag_10m = g_naive - g_det_5m  # Approximate

print(f"\n  C. Growth Rate Decomposition ($10M, $100K ded)")
print(f"     g_naive   = {g_naive*10000:.1f} bps  (analytical, this calculation)")
print(f"     g_det     ≈ {g_det_5m*10000:.1f} bps  (from $5M; nearly identical per doc)")
print(f"     g_actual  = {g_actual_10m*10000:.1f} bps  (from 250K MC simulation)")
print(f"     g_median  = {target['growth_rate_median']*10000:.1f} bps")
print(f"     ─────────────────────────────────────────")
print(f"     Tax Accrual Drag:    ≈{(g_naive - g_det_5m)*10000:>6.1f} bps")
print(f"     Stochastic Penalty:  ≈{(g_det_5m - g_actual_10m)*10000:>6.1f} bps")
print(f"     Total Penalty:        {(g_naive - g_actual_10m)*10000:>6.1f} bps")


In [ ]:

# STEP 6: EBITDA SUMMARY AND WEALTH TRAJECTORY

print("=" * 70)
print("STEP 6: EBITDA ESTIMATE AND WEALTH TRAJECTORY")
print("=" * 70)

print(f"""
┌─────────────────────────────────────────────────────────────────────┐
│  EBITDA ESTIMATE: $10M Cap, ATR=1.0, $100K Deductible (Insured)   │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│  Revenue (Year 1)              $10,000,000                         │
│  × EBITABL Margin                   12.5%                          │
│  ───────────────────────────────────────                           │
│  Gross EBITDA                   $1,250,000                         │
│                                                                     │
│  Less: Insurance Premium         ($356,939)   3.57% of Revenue    │
│  Less: Expected Retained Loss    ($129,181)   1.29% of Revenue    │
│  ───────────────────────────────────────                           │
│  Net EBITDA                       $763,880    7.64% of Revenue    │
│                                                                     │
│  Tax (25%)                       ($190,970)                        │
│  ───────────────────────────────────────                           │
│  Net Income                       $572,910    5.73% of Revenue    │
│                                                                     │
│  Dividends (30%)                 ($171,873)                        │
│  ───────────────────────────────────────                           │
│  Retained Earnings                $401,037    4.01% of Assets     │
│                                                                     │
├─────────────────────────────────────────────────────────────────────┤
│  GROWTH RATE RECONCILIATION                                        │
│                                                                     │
│  Naive g (Ret. Earnings / Assets)    401.0 bps/yr                 │
│  − Tax Accrual Drag                 −171.7 bps/yr  (accrual lag)  │
│  − Stochastic Penalty                −55.3 bps/yr  (σ²/2 + tails)│
│  ───────────────────────────────────────                           │
│  Actual Time-Average Growth Rate     174.0 bps/yr  (MC mean)     │
│  Median Growth Rate                  174.1 bps/yr  ← reported    │
│                                                                     │
├─────────────────────────────────────────────────────────────────────┤
│  50-YEAR WEALTH TRAJECTORY                                         │
│                                                                     │""")

# Terminal wealth calculations
initial = INITIAL_ASSETS
years = 50

# At naive rate
w_naive = initial * np.exp(g_naive * years)
# At actual rate
w_actual_mean = target['final_assets_mean']
w_actual_median = target['final_assets_median']
# At median growth rate
w_from_median_g = initial * np.exp(target['growth_rate_median'] * years)

# No insurance comparison
w_noins_mean = noins['final_assets_mean']
w_noins_median = noins['final_assets_median']

print(f"│  Starting Assets                   ${initial:>12,.0f}                │")
print(f"│                                                                     │")
print(f"│  At Naive Rate (401 bps):          ${w_naive:>12,.0f}   ({w_naive/initial:.2f}×)       │")
print(f"│  Actual Mean Final Assets:         ${w_actual_mean:>12,.0f}   ({w_actual_mean/initial:.2f}×)       │")
print(f"│  Actual Median Final Assets:       ${w_actual_median:>12,.0f}   ({w_actual_median/initial:.2f}×)       │")
print(f"│  From Median g (exp(g×50)):        ${w_from_median_g:>12,.0f}   ({w_from_median_g/initial:.2f}×)       │")
print(f"│                                                                     │")
print(f"│  vs. No Insurance:                                                  │")
print(f"│    No-Ins Mean Final Assets:       ${w_noins_mean:>12,.0f}   ({w_noins_mean/initial:.2f}×)       │")
print(f"│    No-Ins Median Final Assets:     ${w_noins_median:>12,.0f}   ({w_noins_median/initial:.2f}×)       │")
print(f"│    Median Wealth Advantage:        ${w_actual_median - w_noins_median:>12,.0f}   (+{(w_actual_median/w_noins_median - 1)*100:.0f}%)        │")
print(f"│                                                                     │")
print(f"│  Ruin Probability (50yr):                                           │")
print(f"│    Insured:   {target['ruin_probability'][50]:>6.2%}                                           │")
print(f"│    Uninsured: {noins['ruin_probability'][50]:>6.2%}                                          │")
print(f"│                                                                     │")
print(f"└─────────────────────────────────────────────────────────────────────┘")


In [ ]:

# STEP 7: WHY NAIVE g (401 bps) ≠ ACTUAL g (174 bps) - THE TWO PENALTIES

print("=" * 70)
print("STEP 7: EXPLAINING THE 227 bps GAP")
print("=" * 70)

print(f"""
The naive calculation says: "Retain $401K/yr on a $10M base = 4.01% growth."
The simulation says: "Median growth = 1.74%/yr."

Where did the other 227 bps go?

PENALTY 1: Tax Accrual Timing Drag (~172 bps)
──────────────────────────────────────────────
The model uses accrual-basis accounting with a 2-year tax payment lag:
  • Year N: taxes are expensed (reducing book income) AND accrued as liability
  • Year N+2: accrued taxes are paid from CASH

In annual resolution, this means:
  • Retained earnings add to cash:  EBIT × (1−tax) × retention
  • But tax payment from 2 years prior ALSO reduces cash

The net effect: total-asset growth is slower than the simple
  g = margin × (1−tax) × retention formula suggests.

This drag is ~172 bps and affects ALL configurations proportionally.
It does NOT change the ranking between deductible levels.

PENALTY 2: Stochastic Volatility Penalty (~55 bps)
──────────────────────────────────────────────────
For multiplicative processes: g_time ≈ g_expected − σ²/2

Two sources of volatility:
  a) Revenue volatility (GBM σ=0.15):
     Base penalty: σ²/2 = 0.15²/2 = 112.5 bps
     (But modulated by how revenue flows through the P&L)

  b) Retained loss volatility (fat-tailed Pareto losses):
     With $100K deductible, retained losses have:""")

# Load actual retained loss statistics from the simulation
ts_retained = target['ts_retained_losses_pctiles']  # shape (50, 7) for percentiles [5,10,25,50,75,90,95]
yr1_retained_pctiles = ts_retained[0]  # Year 1
pctile_labels = [5, 10, 25, 50, 75, 90, 95]

print(f"     Year 1 Retained Loss Percentiles:")
for pct, val in zip(pctile_labels, yr1_retained_pctiles):
    print(f"       P{pct:2d}: ${val:>12,.0f}  ({val/INITIAL_ASSETS:.2%} of assets)")

print(f"""
  The combination of revenue shocks and retained loss shocks
  creates a ~55 bps drag on the time-average growth rate.

  For comparison, the $0K deductible (GC) has a NEGATIVE penalty
  of −22 bps (insurance actually helps by stabilizing income).

SUMMARY:
  g_naive                    401.0 bps
  − Tax accrual drag        −171.7 bps
  − Stochastic penalty       −55.3 bps
  ─────────────────────────────────────
  g_actual                   174.0 bps  ✓ matches simulation
""")


In [ ]:

# STEP 8: Cross-check with all deductible levels for $10M, ATR=1.0

print("=" * 70)
print("STEP 8: CROSS-CHECK — ALL DEDUCTIBLE LEVELS ($10M, ATR=1.0)")
print("=" * 70)

print(f"\n{'Deductible':<15} {'Mean g':>10} {'Median g':>10} {'Ruin 50yr':>10} {'Median FA':>15}")
print(f"{'─'*15} {'─'*10} {'─'*10} {'─'*10} {'─'*15}")

for k, v in sorted(dc["configs"].items()):
    if v["Cap"] == 10_000_000 and v["ATR"] == 1.0:
        if v.get("NOINS"):
            label = "No Insurance"
        else:
            label = f"${v['Ded']/1000:.0f}K"
        print(f"{label:<15} {v['growth_rate_mean']*10000:>8.1f}bp {v['growth_rate_median']*10000:>8.1f}bp "
              f"{v['ruin_probability'][50]:>9.2%} ${v['final_assets_median']:>13,.0f}")

# Also show $5M for comparison
print(f"\n{'Deductible':<15} {'Mean g':>10} {'Median g':>10} {'Ruin 50yr':>10} {'Median FA':>15}")
print(f"{'─'*15} {'─'*10} {'─'*10} {'─'*10} {'─'*15}")
print("($5M Cap, ATR=1.0 for comparison)")

for k, v in sorted(dc["configs"].items()):
    if v["Cap"] == 5_000_000 and v["ATR"] == 1.0:
        if v.get("NOINS"):
            label = "No Insurance"
        else:
            label = f"${v['Ded']/1000:.0f}K"
        print(f"{label:<15} {v['growth_rate_mean']*10000:>8.1f}bp {v['growth_rate_median']*10000:>8.1f}bp "
              f"{v['ruin_probability'][50]:>9.2%} ${v['final_assets_median']:>13,.0f}")


In [ ]:
PAPER_OUTPUT =  Path('output/publication/interesting')
PAPER_OUTPUT.mkdir(parents=True, exist_ok=True)

# Copy interesting graphs to the "interesting" folder
import shutil
interesting_figs = [
    'insurance_outcome_distribution_basic.png',
    'volatility_tax_vs_premium_savings.png',
    'wealth_fan_chart.png',
    'year_by_year_growth_lift_facets.png',
]
for fig_name in interesting_figs:
    try:
        src = Path('output/publication') / fig_name
        dst = PAPER_OUTPUT / fig_name
        shutil.copy2(src, dst)
    except Exception as e:
        print(f'Error copying {fig_name}: {e}')